## MarTech Challenge 用户购买预测思路整理（二分类）

In [5]:
import pandas as pd
raw = pd.read_csv('/home/guanqzha/Data/train.csv')

raw.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2306871 entries, 0 to 2306870
Data columns (total 29 columns):
order_detail_id           2306871 non-null int64
order_id                  2306871 non-null int64
order_total_num           2306871 non-null float64
order_amount              2306871 non-null float64
order_total_payment       2306871 non-null float64
order_total_discount      2306871 non-null float64
order_pay_time            2306871 non-null object
order_status              2306871 non-null int64
order_count               2306871 non-null float64
is_customer_rate          2306871 non-null float64
order_detail_status       2306871 non-null float64
order_detail_goods_num    2306871 non-null float64
order_detail_amount       2306871 non-null float64
order_detail_payment      2306871 non-null float64
order_detail_discount     2306871 non-null float64
customer_province         2305732 non-null object
customer_city             2305721 non-null object
member_id                 230

In [9]:
raw.describe()

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_status,order_count,is_customer_rate,order_detail_status,...,member_id,customer_id,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount
count,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,2.306871e+06,...,2.306871e+06,2.306871e+06,635790.000000,635790.0,635790.0,2.306871e+06,2.306871e+06,2.306435e+06,2.306871e+06,2.306871e+06
mean,2.340043e+06,2.136152e+06,1.684285e+00,3.108007e+02,1.114688e+02,5.908662e+00,9.858623e+00,1.767748e+00,3.141311e-02,1.035626e+01,...,5.789832e+05,1.933233e+06,0.079095,1.0,1.0,1.558213e+03,1.558213e+03,5.219550e+01,1.091712e+00,8.613616e-02
std,7.740314e+05,6.500690e+05,3.941060e+00,4.040238e+02,1.644054e+02,1.711463e+01,1.922945e+01,4.800121e+02,1.744315e-01,2.027679e+01,...,1.075381e+06,5.538885e+05,0.361848,0.0,0.0,7.803288e+02,7.803288e+02,6.325725e+00,3.937307e-01,2.805651e-01
min,1.000000e+06,1.000000e+06,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,1.000000e+06,0.000000,1.0,1.0,9.980000e+02,9.980000e+02,-7.997479e+01,0.000000e+00,0.000000e+00
25%,1.669684e+06,1.571484e+06,1.000000e+00,1.990000e+02,6.690000e+01,0.000000e+00,6.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,...,0.000000e+00,1.436434e+06,0.000000,1.0,1.0,1.038000e+03,1.038000e+03,4.960044e+01,1.000000e+00,0.000000e+00
50%,2.339350e+06,2.142988e+06,1.000000e+00,2.399000e+02,8.990000e+01,0.000000e+00,6.000000e+00,1.000000e+00,0.000000e+00,6.000000e+00,...,0.000000e+00,1.929569e+06,0.000000,1.0,1.0,1.233000e+03,1.233000e+03,5.390926e+01,1.000000e+00,0.000000e+00
75%,3.007430e+06,2.707954e+06,2.000000e+00,3.398000e+02,1.131700e+02,5.000000e+00,6.000000e+00,2.000000e+00,0.000000e+00,6.000000e+00,...,4.834840e+05,2.457816e+06,0.000000,1.0,1.0,1.643000e+03,1.643000e+03,5.485061e+01,1.000000e+00,0.000000e+00
max,3.685499e+06,3.238361e+06,1.700000e+03,1.023740e+05,9.840150e+04,1.960000e+03,1.010000e+02,7.290610e+05,1.000000e+00,1.010000e+02,...,4.161555e+06,2.826574e+06,2.000000,1.0,1.0,6.673000e+03,6.673000e+03,9.235260e+01,2.000000e+00,1.000000e+00


In [10]:
raw.head(10)

,order_detail_id,order_id,order_total_num,order_amount,order_total_payment,order_total_discount,order_pay_time,order_status,order_count,is_customer_rate,...,customer_gender,member_status,is_member_actived,goods_id,goods_class_id,goods_price,goods_status,goods_has_discount,goods_list_time,goods_delist_time
0,1000000,1000000,1.0,239.9,96.9,0.0,2012-11-01 00:10:56,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.909289,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
1,1001530,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1953,1953,45.961352,0.0,1.0,2013-08-28 17:27:50,2013-09-01 00:38:17
2,1001531,1001327,2.0,288.0,96.9,0.0,2013-08-31 23:14:42,6,2.0,0.0,...,NaN,NaN,NaN,1083,1083,53.035439,1.0,0.0,2014-10-29 18:21:05,2014-11-05 18:21:05
3,1001532,1001328,3.0,180.0,89.7,0.0,2013-08-31 22:06:35,6,1.0,0.0,...,NaN,NaN,NaN,1013,1013,46.046917,1.0,1.0,2014-10-25 11:00:00,2014-11-01 11:00:00
4,1001533,1001329,1.0,159.9,65.9,0.0,2013-08-31 21:33:36,6,1.0,0.0,...,NaN,NaN,NaN,1628,1628,50.722161,1.0,0.0,2014-10-23 15:35:33,2014-10-30 15:35:33
5,1001534,1001330,1.0,129.9,66.9,0.0,2013-08-31 20:57:08,6,1.0,0.0,...,NaN,NaN,NaN,1038,1038,49.408291,1.0,0.0,2014-10-28 14:26:28,2014-11-04 14:26:28
6,1001535,1001331,1.0,239.9,89.9,0.0,2013-08-31 20:21:05,6,1.0,0.0,...,NaN,NaN,NaN,998,998,54.850109,1.0,0.0,2014-10-25 11:08:07,2014-11-01 11:08:07
7,1001536,1001332,1.0,289.9,115.9,0.0,2013-08-31 19:49:50,6,1.0,0.0,...,NaN,NaN,NaN,1858,1858,56.745159,1.0,0.0,2014-10-24 23:59:59,2014-10-31 23:59:59
8,1001537,1001333,1.0,159.9,59.9,0.0,2013-08-31 19:11:24,6,1.0,0.0,...,NaN,NaN,NaN,1943,1943,50.808977,1.0,0.0,2014-10-24 00:00:00,2014-10-31 00:00:00
9,1001538,1001334,1.0,129.9,66.9,0.0,2013-08-31 18:31:05,6,1.0,0.0,...,NaN,NaN,NaN,1038,1038,49.592109,1.0,0.0,2014-10-28 14:26:28,2014-11-04 14:26:28


----
- missing data varies in [customer_gender]  [member_status] [is_member_actived]
- [customer_province] [customer_city] [goods_prices] in some minor lack value
----
### data statistics

In [6]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
raw["customer_id"].value_counts(bins=100)

(2753511.04, 2771776.78]    29826
(2607385.12, 2625650.86]    29469
(2680448.08, 2698713.82]    29424
(2461259.2, 2479524.94]     29298
(2516056.42, 2534322.16]    29157
                            ...  
(2223804.58, 2242070.32]    18508
(2260336.06, 2278601.8]     18320
(2296867.54, 2315133.28]    18259
(2333399.02, 2351664.76]    18004
(2369930.5, 2388196.24]     17918
Name: customer_id, Length: 100, dtype: int64

In [ ]:
import datetime
import numpy as np

# 对数据进行预处理，各种特征提取
def preprocess(raw, train='train'):
    # 对性别为空的情况，进行0填充
    data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
    # 添加新的列：商品相关（最后一次行为）
    data[['goods_id_last', 'goods_status_last', 'goods_price_last', 'goods_has_discount_last',           'goods_list_time_last', 'goods_delist_time_last']] = raw.groupby('customer_id')['goods_id',             'goods_status', 'goods_price', 'goods_has_discount', 'goods_list_time', 'goods_delist_time'].last()
    data[['order_total_num_last', 'order_amount_last', 'order_total_payment_last', 'order_total_discount_last', 'order_pay_time_last',           'order_status_last', 'order_count_last', 'is_customer_rate_last', 'order_detail_status_last', 'order_detail_goods_num_last',           'order_detail_amount_last', 'order_detail_payment_last', 'order_detail_discount']] = raw.groupby('customer_id')['order_total_num',         'order_amount', 'order_total_payment', 'order_total_discount', 'order_pay_time', 'order_status', 'order_count', 'is_customer_rate',         'order_detail_status', 'order_detail_goods_num', 'order_detail_amount', 'order_detail_payment', 'order_detail_discount'].last()
    # 添加商品原始价格（多种统计字段） goods_price： mean, min, max, std
    data[['goods_price_mean', 'goods_price_min', 'goods_price_max', 'goods_price_std']] =         raw.groupby('customer_id')['goods_price'].agg([
            ('goods_price_mean', 'mean'),
            ('goods_price_min', 'min'),
            ('goods_price_max', 'max'),
            ('goods_price_std', 'std')])
   # 订单实付金额（多种统计字段） order_detail_payment： mean, min, max, std
    data[['order_detail_payment_mean', 'order_detail_payment_min', 'order_detail_payment_max', 'order_detail_payment_std']] =         raw.groupby('customer_id')['order_detail_payment'].agg([
            ('order_detail_payment_mean', 'mean'),
            ('order_detail_payment_min', 'min'),
            ('order_detail_payment_max', 'max'),
            ('order_detail_payment_std', 'std')])
    # 用户购买的订单数量
    data['count'] = raw.groupby('customer_id')['order_id'].nunique()
    # 用户购买的商品数量
    #data['goods_count'] = raw.groupby('customer_id')['goods_id'].nunique()
    # order_total_num
    data['order_total_sum'] = raw.groupby('customer_id')['order_total_num'].sum()
    # 用户所在省份
    data['customer_province'] = raw.groupby('customer_id')['customer_province'].last()
    # 用户所在城市
    data['customer_city'] = raw.groupby('customer_id')['customer_city'].last()
    # 用户是否评价 统计结果（平均，总和）
    data[['is_customer_rate_ratio', 'is_customer_rate_sum']] = raw.groupby('customer_id')['is_customer_rate'].agg([
        ('is_customer_rate_ratio', np.mean),
        ('is_customer_rate_sum', np.sum)
    ])
    # 用户购买的goods数量
    data['order_detail_total_num'] = raw.groupby('customer_id')['order_detail_goods_num'].sum()
    # 商品折扣统计属性（sum, mean）
    data[['goods_has_discount_sum', 'goods_has_discount_mean']] = raw.groupby('customer_id')['goods_has_discount'].agg([
        ('goods_has_discount_sum', np.sum),
        ('goods_has_discount_mean', np.mean)
    ])
    # 订单实付金额 统计属性（sum, mean）
    data[['order_total_payment_sum', 'order_total_payment_mean']] = raw.groupby('customer_id')['order_total_payment'].agg([
        ('order_total_payment_sum', np.sum),
        ('order_total_payment_mean', np.mean)
    ])
    # 时间转换
    def time2multi(x):
        t = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        # 2013-01-01 的day = 1, weekday = 星期二（2-1=1）
        return pd.Series([t.month, t.day, t.weekday(), t.hour])
    
    # 起始时间 2013-01-01
    t_str = '2013-01-01 00:00:00'
    t = datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
    
    # 商品最新上架时间diff（假设起始时间为2013-01-01 00:00:00）
    data['goods_list_time_diff'] = data['goods_list_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    # 商品最新下架时间diff（假设起始时间为2013-01-01 00:00:00）
    data['goods_delist_time_diff'] = data['goods_delist_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    # 商品展示时间
    data['goods_diff'] = data['goods_delist_time_diff'] - data['goods_list_time_diff']
    
    # 付款时间的尺度
    data[['order_pay_time_last_m', 'order_pay_time_last_d', 'order_pay_time_last_week', 'order_pay_time_last_h']] =         data['order_pay_time_last'].apply(time2multi)
    data['order_pay_time_last_diff'] = data['order_pay_time_last'].map(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') - t).days/365)
    return data


# In[46]:


get_ipython().run_cell_magic('time', '', "# 8月之前的做训练集，8月的数据做验证集\nraw['order_pay_time'].max()\ntrain_raw = raw[raw['order_pay_time'] <= '2013-07-31 23:59:59']\n# 提取训练集 各种特征\ntrain_raw = preprocess(train_raw)\ntrain_raw")


# In[47]:


label_raw = set(raw[raw['order_pay_time'] > '2013-07-31 23:59:59']['customer_id'].dropna())
# 如果这个用户在8月份进行了购买 label=1，否则label=0
# 只有用户有order 就会有行为
train_raw['label'] = train_raw.index.map(lambda x:int(x in label_raw))


# In[48]:


get_ipython().run_cell_magic('time', '', '# 用全量的用户特征进行预测，raw 即9月份之前的所有数据\ntest = preprocess(raw)')


# In[50]:


import pickle
with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)
with open('train_raw.pkl', 'wb') as file:
    pickle.dump(train_raw, file)


# In[53]:


# 有些时间戳提取了时间尺度，需要去掉
train_data = train_raw.drop(['goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last'], axis=1)


# In[57]:


train_data['customer_province'].fillna('0', inplace=True)
train_data['customer_city'].fillna('0', inplace=True)


# In[76]:


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['customer_province'] = le.fit_transform(train_data['customer_province'])
train_data['customer_city'] = le.fit_transform(train_data['customer_city'])
train_data = train_data.drop(['customer_province', 'customer_city'], axis=1)


# In[77]:


# 分类变量
cate1 = ['goods_id_last', 'goods_status_last', 'order_status_last',         'customer_gender', 'order_detail_status_last',         'order_pay_time_last_h', 'order_pay_time_last_week', 'order_pay_time_last_h']


# In[73]:


#train_data[['customer_province', 'customer_city']].isnull().sum()
train_data['customer_province'] = train_data['customer_province'].astype('int64')
train_data['customer_city'] = train_data['customer_city'].astype('int64')


# In[78]:


train_data.info()


# In[79]:


# 训练集切分
from sklearn.model_selection import KFold
import lightgbm as lgb

# 采用CV=5折交叉验证
kf = KFold(n_splits=5,shuffle=True)
y_pred = 0
for train_index , test_index in kf.split(train_data):
    # 设置每一折的 train和test
    X_train, X_valid, y_train, y_valid = train_data.drop('label', axis=1).iloc[train_index], train_data.drop('label', axis=1).iloc[test_index],         np.array(train_data['label'])[train_index], np.array(train_data['label'])[test_index]
    param = {
        'num_leaves': 121,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'learning_rate': 0.05,
        'metric': 'binary_logloss'
    }
    # 使用lgb进行训练
    trn_data = lgb.Dataset(X_train, y_train)
    val_data = lgb.Dataset(X_valid, y_valid)
    lgbm = lgb.train(param, trn_data, valid_sets=[trn_data, val_data], num_boost_round=10000, early_stopping_rounds=100, verbose_eval=50, categorical_feature=cate1)
    # 计算test的预测结果
    test = test[X_train.columns]
    y_pred = y_pred + lgbm.predict(test)*0.2
    


# In[82]:


pd.DataFrame({
    'column': X_train.columns,
    'importance':lgbm.feature_importance()
}).sort_values(by='importance')


# In[88]:


def f(x):
    """
    if x<0.1:
        return 0.1
    if x>0.9:
        return 0.9
    """
    return round(x*1000)/1000

test['result'] = y_pred
test['result'] = test['result'].map(f)
result = pd.DataFrame(test['result'])
result.to_csv('submission5.csv')
result


# In[96]:


temp = result.reset_index()
temp['result'] = temp['result'].map(lambda x: 0 if x<0.5 else 1)
temp
temp = temp.sort_values(by='customer_id')
temp.to_csv('./test2.csv', index=False)